In [82]:
import numpy as np
import pandas as pd
from pandas import Series, DataFrame
import seaborn as sns
import matplotlib.pyplot as plt

pd.set_option('display.max_columns', 150)
pd.set_option('display.max_rows', 30)

%matplotlib inline

In [70]:
import statsmodels.api as sm

In [141]:
dfa = dframe[['age','gender','ethnicity','unittype','expiremarker','ventmarker','septicflag'
               ,'anyacuteischemiccondition','activecardiacischemia','apachescore'
               ,'cirrhosisflag','ckd','chf','vasopressor','hgbmin','transfusionmarker']]
dfa=dfa.dropna()

In [165]:
dfa.describe()

,age,expiremarker,ventmarker,septicflag,anyacuteischemiccondition,activecardiacischemia,apachescore,cirrhosisflag,ckd,chf,vasopressor,hgbmin,transfusionmarker
count,29207.000000,29207.000000,29207.000000,29207.000000,29207.000000,29207.000000,29207.000000,29207.000000,29207.000000,29207.000000,29207.000000,29207.000000,29207.000000
mean,64.004383,0.072003,0.388537,0.171055,0.053754,0.117575,57.527339,0.007978,0.064060,0.087068,0.186359,10.201455,0.042558
std,16.832691,0.258498,0.487426,0.376563,0.225536,0.322109,27.181467,0.088962,0.244864,0.281940,0.389403,2.418884,0.201863
min,7.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-1.000000,0.000000,0.000000,0.000000,0.000000,1.600000,0.000000
25%,54.000000,0.000000,0.000000,0.000000,0.000000,0.000000,39.000000,0.000000,0.000000,0.000000,0.000000,8.300000,0.000000
50%,66.000000,0.000000,0.000000,0.000000,0.000000,0.000000,53.000000,0.000000,0.000000,0.000000,0.000000,10.100000,0.000000
75%,77.000000,0.000000,1.000000,0.000000,0.000000,0.000000,71.000000,0.000000,0.000000,0.000000,0.000000,11.900000,0.000000
max,89.000000,1.000000,1.000000,1.000000,1.000000,1.000000,211.000000,1.000000,1.000000,1.000000,1.000000,20.800000,1.000000


In [148]:
X = dfa.drop(['expiremarker','transfusionmarker'],1)
dummy_X = pd.get_dummies(X[['gender','ethnicity','unittype']],drop_first=True)


X2=pd.concat([X, dummy_X], axis=1, join_axes=[X.index])
X = X2.drop(['gender','ethnicity','unittype'],1)

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: FutureWarning: The join_axes-keyword is deprecated. Use .reindex or .reindex_like on the result to achieve the same functionality.
  """


In [149]:
ps_target = dfa['transfusionmarker']
dummy_ps = pd.get_dummies(ps_target)
y = dummy_ps[1]
y

0        1
1        1
2        1
3        1
4        1
        ..
29141    0
29142    0
29143    0
29144    0
29145    0
Name: 1, Length: 29207, dtype: uint8

In [163]:
glm = sm.Logit(y, X)
result = glm.fit()
ps = pd.DataFrame(result.predict(X))

         Current function value: inf
         Iterations: 35


/anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:512: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


In [164]:
ps

,0
0,0.310553
1,0.271816
2,0.161748
3,0.196729
4,0.081753
...,...
29141,0.001162
29142,0.023409
29143,0.045762
29144,0.010337


### PS Marching

In [171]:
z = dfa['expiremarker']
dummy_y = pd.get_dummies(z,drop_first=True)
dummy_y

,1
0,0
1,0
2,0
3,0
4,0
...,...
29141,0
29142,0
29143,0
29144,0


In [172]:
dummy_y.describe()

,1
count,29207.000000
mean,0.072003
std,0.258498
min,0.000000
25%,0.000000
50%,0.000000
75%,0.000000
max,1.000000


In [173]:
ps = pd.Series(result.predict(X))
z1 = y
y= dummy_y

In [174]:
table = pd.concat([ps,z1,y],axis=1)
table.columns = ['ps','treat','died']

In [175]:
table

,ps,treat,died
0,0.310553,1,0
1,0.271816,1,0
2,0.161748,1,0
3,0.196729,1,0
4,0.081753,1,0
...,...,...,...
29141,0.001162,0,0
29142,0.023409,0,0
29143,0.045762,0,0
29144,0.010337,0,0


In [ ]:
interval = np.arange(0,1.01,0.01)
match_list = []
for i in range(0,len(interval)-1):
    temp0 = table[(table['treat']==0) & (interval[i] < table['ps']) & (table['ps'] < interval[i+1])]
    temp1 = table[(table['treat']==1) & (interval[i] < table['ps']) & (table['ps'] < interval[i+1])]
    if (len(temp0) > 0) & (len(temp1) > 0):
        match_list.append(temp1['died'].mean()-temp0['died'].mean())

np.mean(match_list)